In [ ]:
%pip install mcp --quiet --upgrade

In [ ]:
from genscai.tools import mcp

mcp.run()

In [1]:
from genscai.tools import MCPClient

mcp_client = MCPClient()

In [2]:
mcp_client.connect("mcp/server.py")

RuntimeError: Cannot run the event loop while another loop is running

In [ ]:
tools = await mcp_client.get_tools()

In [ ]:
for tool in tools:
    print(f"Tool Name: {tool}")

In [ ]:
from IPython.display import Code

Code(filename="../mcp/server.py", language="python")

In [ ]:
!mcp run ../mcp/server.py

In [ ]:
!npx @modelcontextprotocol/inspector \
    uv \
    --directory path/to/server \
    run \
    package-name \
    args...

In [ ]:
import subprocess

server = subprocess.Popen(
    ["python3", "server.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    stdin=subprocess.PIPE,
    text=True,
)

In [ ]:
import json


def create_message(method_name, params, id=None):
    message = {"jsonrpc": "2.0", "method": method_name, "params": params, "id": id}
    return json.dumps(message)


def send_message(message):
    server.stdin.write(message + "\n")
    server.stdin.flush()


def receive_message():
    server_output = json.loads(server.stdout.readline())
    if "result" in server_output:
        return server_output["result"]
    else:
        return "Error"

In [ ]:
id = 1
init_message = create_message(
    "initialize",
    {
        "clientInfo": {"name": "Llama Agent", "version": "0.1"},
        "protocolVersion": "2024-11-05",
        "capabilities": {},
    },
    id,
)

send_message(init_message)
response = receive_message()
server_name = response["serverInfo"]["name"]
print("Initializing  " + server_name + "...")

init_complete_message = create_message("notifications/initialized", {})
send_message(init_complete_message)
print("Initialization complete.")

In [ ]:
id += 1
list_tools_message = create_message("tools/list", {}, id)
send_message(list_tools_message)
response = json.loads(server.stdout.readline())["result"]
for tool in response["tools"]:
    print(tool["name"])
    print(tool["description"])
    print(tool["inputSchema"]["properties"])
    print("")

In [ ]:
available_functions = []
for tool in response["tools"]:
    func = {
        "type": "function",
        "function": {
            "name": tool["name"],
            "description": tool["description"],
            "parameters": {
                "type": "object",
                "properties": tool["inputSchema"]["properties"],
                "required": tool["inputSchema"]["required"],
            },
        },
    }
    available_functions.append(func)

## References
- https://medium.com/predict/using-the-model-context-protocol-mcp-with-a-local-llm-e398d6f318c3
- https://modelcontextprotocol.io/docs/tools/inspector#python
- https://github.com/modelcontextprotocol/python-sdk